## Jul 15, 2022

# Enhance image-depth GT / pano-depth GT

### Image-depth GT
- Add PanoDepth visibility mask 
- Add yaw angles per points

### Pano-depth GT
- Add yaw angles per points

In [ ]:
%load_ext autoreload
%autoreload 2

import copy
import os
import torch
import numpy as np
import pythreejs as pjs
import warnings
warnings.filterwarnings('ignore')

from matplotlib.cm import get_cmap
from tqdm.notebook import tqdm
from PIL import Image
from IPython.core.display import display

os.chdir('..')
np.set_printoptions(precision=4)
!pwd

# Load batch

In [ ]:
from vidar.utils.config import read_config

config = read_config('configs/papers/panodepth/train_ddad.yaml')

# Resize depth for easy debugging
# config.datasets.train.augmentation.resize_supervision = True
# config.datasets.train.dataloader.num_workers = 0
config.datasets.validation.labels += ['lidar']
config.datasets.validation.dataloader.batch_size = 1
config.datasets.validation.dataloader.num_workers = 0

In [ ]:
from vidar.utils.setup import setup_dataset, setup_dataloader

# dataset = setup_dataset(config.datasets.train, verbose=True)
# dataloader = setup_dataloader(dataset, config.datasets.train.dataloader, 'train')

dataset = setup_dataset(config.datasets.validation, verbose=True)
dataloader = setup_dataloader(dataset, config.datasets.validation.dataloader, 'val')

In [ ]:
batch_from_loader = next(iter(dataloader[0]))
batch_from_loader.keys()

## Utilities

In [ ]:
def to_numpy(tensor):
    return tensor.permute(1, 2, 0).numpy()

def to_uint8(array):
    return (array * 255.0).astype(np.uint8)

In [ ]:
import pythreejs as pjs

def visualize_3d(xyz, rgb=None, size=0.03, height=480, width=480):
    points_buf = pjs.BufferAttribute(array=xyz)
    geometryAttrs = {'position': points_buf}

    if rgb is not None:
        colors_buf = pjs.BufferAttribute(array=rgb)
        geometryAttrs['color'] = colors_buf
    
    geometry = pjs.BufferGeometry(attributes=geometryAttrs)

    material = pjs.PointsMaterial(vertexColors='VertexColors', size=size)
    pointCloud = pjs.Points(geometry=geometry, material=material)

    pythreejs_camera = pjs.PerspectiveCamera(    
        up=[1, 0, 1],
        children=[pjs.DirectionalLight(color='white', intensity=0.5)])

    pythreejs_camera.rotateX(np.pi/4)
    pythreejs_camera.position = (-15., 0., 30.)

    scene = pjs.Scene(children=[
                    pointCloud,
                    pythreejs_camera,
                    pjs.AmbientLight(color='#777777')])
    
    axes = pjs.AxesHelper(size=3)
    scene.add(axes)
        
    control = pjs.OrbitControls(controlling=pythreejs_camera)
    renderer = pjs.Renderer(camera=pythreejs_camera, 
                        scene=scene, 
                        width=width,
                        height=height,
                        preserveDrawingBuffer=True,
                        controls=[control])
    
    return renderer

# Calculate yaw angles (image-depth)

# Calculate yaw angles (pano-depth)